Gökçe Şahin
090190306
sahingo19@itu.edu.tr

In [73]:
import pandas as pd
import wbgapi as wb
import numpy as np
import statsmodels.api as sm
import yfinance as yf

from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statsmodels.tools as tools

from statsmodels.formula.api import logit
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Question 1

For this question use the World Bank Data for Turkey for the following indicators. Use [wbgapi](https://pypi.org/project/wbgapi/) for getting the data.

* [Literacy rate, adult female (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
* [Labor force, female (SL.TLF.TOTL.FE.ZS)](https://data.worldbank.org/indicator/SL.TLF.TOTL.FE.ZS)
* [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
* [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
* [GDP per capita (NY.GDP.PCAP.CD)](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
* [Mortality rate, under-5 (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)


Using the [statsmodels](https://www.statsmodels.org/stable/index.html) library write the best linear regression model using child mortality as the dependent variable while the rest are considered as independent variables. Pay particular attention to the fact that the order of the variables put into the model significantly impacts the performance of the model. Choose the best model by considering

* with the minimum number of variables and their interactions,
* with the optimal ordering of the independent variables and their interactions,
* $R^2$-score of the model,
* statistical significance of the model coefficients,
* ANOVA analysis of the model.


In [74]:
def extract(df, cntry, name):
    tmp = df[['time','value']][df['economy'] == cntry]
    tmp.index = tmp.time
    del tmp['time']
    tmp.columns = [[name]]
    return tmp

In [75]:
def vars_mor(cntry, dfs, vars):
    arr = [extract(dfs[i], cntry, vars[i]) for i in range(len(vars))]
    res = arr[0].join(arr[1:6])
    res.dropna(inplace=True)
    
    return res

In [76]:
var_apis = {
    'literacy': 'SE.ADT.LITR.FE.ZS',
    'labor': 'SL.TLF.TOTL.FE.ZS',
    'poverty': 'SI.POV.NAHC',
    'health': 'SH.XPD.CHEX.PC.CD',
    'GDP': 'NY.GDP.PCAP.CD',
    'mortality': 'SH.DYN.MORT'
}


In [77]:
vars = ['literacy', 'labor', 'mortality']
# independent variables + dependent variable (mortality)

In [78]:
dfs = [pd.DataFrame(list(wb.data.fetch(api))) for api in [var_apis[var] for var in vars]]

In [79]:
res = vars_mor('TUR', dfs, vars)
model = ols('mortality ~ literacy + labor', data=res).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              mortality   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     252.1
Date:                Fri, 04 Nov 2022   Prob (F-statistic):           1.58e-10
Time:                        04:05:44   Log-Likelihood:                -33.794
No. Observations:                  15   AIC:                             73.59
Df Residuals:                      12   BIC:                             75.71
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    147.9560      8.876     16.669      0.0

C:\Users\sahin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# Question 2

For this question use Yahoo's Finance API for the following tickers:

* Gold futures (GC=F)
* Silver futures (SI=F)
* Copper futures (HG=F)
* Platinum futures (PL=F)

1. Write the best linear regression model that explains gold futures closing prices in terms of opening prices of gold, silver, copper, and platinum futures.
2. Repeat the same for silver, copper and platinum prices.
3. Compare the models you obtained in Steps 1 and 2. Which model is better? How do you decide? Explain.

In [80]:
def tickers_openning(x, ftickers, fticker_names):
    tmp = {}
    tmp['GCF_o'] = ftickers[0]['Open']
    tmp['SIF_o'] = ftickers[1]['Open']
    tmp['HGF_o'] = ftickers[2]['Open']
    tmp['PLF_o'] = ftickers[3]['Open']
    tmp[x[:-2]+x[-1]+'_c'] = ftickers[fticker_names.index(x)]['Close']

    data = pd.DataFrame(tmp).dropna()

    return data

In [81]:
fticker_names = ['GC=F', 'SI=F', 'HG=F', 'PL=F']
ftickers = [yf.download(ticker) for ticker in fticker_names]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [82]:
[gcfc, sifc, hgfc, plfc] = [tickers_openning(x, ftickers, fticker_names) for x in fticker_names]
gcfc


,GCF_o,SIF_o,HGF_o,PLF_o,GCF_c
Date,,,,,
2000-08-30,273.899994,4.950000,0.8790,593.900024,273.899994
2000-08-31,274.799988,4.920000,0.8850,589.000000,278.299988
2000-09-01,277.000000,5.035000,0.8780,588.000000,277.000000
2000-09-05,275.799988,4.990000,0.8960,602.000000,275.799988
2000-09-06,274.200012,5.000000,0.9050,603.000000,274.200012
...,...,...,...,...,...
2022-10-28,1661.000000,19.500000,3.5375,958.400024,1639.599976
2022-10-31,1641.800049,19.200001,3.4095,939.400024,1635.900024
2022-11-01,1630.800049,19.125000,3.4945,959.799988,1645.000000


### Linear regression models that explains GC = F

In [83]:
model = ols('GCF_c ~ SIF_o + HGF_o + PLF_o', data=gcfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GCF_c   R-squared:                       0.878
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                 1.167e+04
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:05:46   Log-Likelihood:                -32171.
No. Observations:                4863   AIC:                         6.435e+04
Df Residuals:                    4859   BIC:                         6.438e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    431.4329      8.454     51.031      0.000     414.859     448.007
SIF_o         49.1484      0.617     79.612      0.000      47.938      50.359
HGF_o        228.9829      4.671     49.027      0.000     219.827     238.139
PLF_o         -0.6944      0.011    -61.117      0.000      -0.717      -0.672
==============================================================================
Omnibus:                      375.788   Durbin-Watson:                   0.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              873.417
Skew:                          -0.478   Prob(JB):                    2.19e-190
Kurtosis:                       4.843   Cond. No.                     3.78e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains SI = F

In [84]:
model = ols('SIF_c ~ GCF_o + HGF_o + PLF_o', data=sifc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  SIF_c   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.898
Method:                 Least Squares   F-statistic:                 1.431e+04
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:05:46   Log-Likelihood:                -11839.
No. Observations:                4863   AIC:                         2.369e+04
Df Residuals:                    4859   BIC:                         2.371e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.5911      0.129    -50.995      0.000      -6.844      -6.338
GCF_o          0.0115      0.000     79.578      0.000       0.011       0.012
HGF_o         -0.4496      0.087     -5.167      0.000      -0.620      -0.279
PLF_o          0.0109      0.000     63.802      0.000       0.011       0.011
==============================================================================
Omnibus:                     1298.984   Durbin-Watson:                   0.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6387.656
Skew:                           1.200   Prob(JB):                         0.00
Kurtosis:                       8.076   Cond. No.                     5.44e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains HG = F

In [85]:
model = ols('HGF_c ~ GCF_o + SIF_o + PLF_o', data=hgfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  HGF_c   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     8058.
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:05:46   Log-Likelihood:                -3072.0
No. Observations:                4863   AIC:                             6152.
Df Residuals:                    4859   BIC:                             6178.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3247      0.026    -12.492      0.000      -0.376      -0.274
GCF_o          0.0015   2.96e-05     49.002      0.000       0.001       0.002
SIF_o         -0.0129      0.002     -5.452      0.000      -0.017      -0.008
PLF_o          0.0015   3.18e-05     46.061      0.000       0.001       0.002
==============================================================================
Omnibus:                      645.687   Durbin-Watson:                   0.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              956.330
Skew:                           0.974   Prob(JB):                    2.17e-208
Kurtosis:                       3.961   Cond. No.                     6.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Linear regression models that explains PL = F

In [86]:
model = ols('PLF_c ~ GCF_o + SIF_o + HGF_o', data=plfc).fit()
display(model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PLF_c   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     5761.
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:05:46   Log-Likelihood:                -31920.
No. Observations:                4863   AIC:                         6.385e+04
Df Residuals:                    4859   BIC:                         6.387e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    524.4145      6.522     80.405      0.000     511.628     537.201
GCF_o         -0.6274      0.010    -61.218      0.000      -0.647      -0.607
SIF_o         41.9356      0.657     63.790      0.000      40.647      43.224
HGF_o        207.9819      4.527     45.947      0.000     199.108     216.856
==============================================================================
Omnibus:                      247.087   Durbin-Watson:                   0.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              326.933
Skew:                           0.493   Prob(JB):                     1.02e-71
Kurtosis:                       3.801   Cond. No.                     3.48e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

***
Linear regression model that explains SI=F closing prices is better than the others because its R^2 value is larger.
***

# Question 3

1. Write a function that takes a ticker symbol and returns a pandas dataframe that for each day puts a 1 when the closing price is higher than the opening price, a 0 when the closing price is lower than the opening price.
2. Write the best logistic regression that predicts the time series you obtain from Step 1 for gold futures against the opening prices of gold, silver, copper, and platinum prices.
3. Repeat the same for silver, copper, and platinum prices.
4. Compare the models you obtained from Steps 2 and 3. Decide which is the best model, and explain your reasoning.
5. Does any of the models provide a good fit? Explain.

In [87]:
def openning_closing(ticker):
    new_header = ticker[:-2]+ticker[-1]+'_co'

    tmp = yf.download(ticker)
    tmp[new_header] = 0

    tmp[new_header][tmp['Close'] > tmp['Open']] = 1

    tmp_co = pd.DataFrame(tmp[new_header])

    return tmp_co

In [88]:
def ticker_co_df(ticker, tickers_df):
    df = pd.concat([tickers_df, openning_closing(ticker)], axis = 1)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    del df['Date']
    return df


In [89]:
def bootstrap(X, y, model):
    res = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)
        model.fit(X_train,y_train)
        res.append(model.score(X_test,y_test))
    tmp = sorted(res)[3:97]
    return (min(tmp),max(tmp))

In [90]:
def print_model(df):
    X = df.iloc[:, :4]
    y = df.iloc[:, 5]

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

    model = LogisticRegression(max_iter=1500)
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    y_predict = model.predict(X_test)
    print(confusion_matrix(y_test, y_predict))

    print(bootstrap(X, y, model))


In [91]:
fticker_names = ['GC=F', 'SI=F', 'HG=F', 'PL=F']
ftickers = [yf.download(ticker) for ticker in fticker_names]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [92]:
# I used tickers_openning() function from question-2
gcf_df = tickers_openning('GC=F', ftickers, fticker_names)
sif_df = tickers_openning('SI=F', ftickers, fticker_names)
hgf_df = tickers_openning('HG=F', ftickers, fticker_names)
plf_df = tickers_openning('PL=F', ftickers, fticker_names)

In [93]:
model = ticker_co_df('GC=F', gcf_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.5822368421052632
[[615  87]
 [421  93]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_10336\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.553453947368421, 0.6019736842105263)


In [94]:
model = ticker_co_df('SI=F', sif_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.65625
[[794   4]
 [414   4]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_10336\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.6356907894736842, 0.6817434210526315)


In [95]:
model = ticker_co_df('HG=F', hgf_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.5238486842105263
[[637   0]
 [579   0]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_10336\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.48273026315789475, 0.5263157894736842)


In [96]:
model = ticker_co_df('PL=F', plf_df)
print_model(model)

[*********************100%***********************]  1 of 1 completed
0.712171052631579
[[796  81]
 [269  70]]


C:\Users\sahin\AppData\Local\Temp\ipykernel_10336\4014248705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[new_header][tmp['Close'] > tmp['Open']] = 1


(0.7088815789473685, 0.75)


# Question 4

For this question use the following [data](https://archive.ics.uci.edu/ml/datasets/credit+approval):


In [97]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)

fn = {'+': 1, '-': 0}

X = credit.replace('?',0).iloc[:,[1,2,7,10,14]]
y = credit.iloc[:,15].map(lambda x: fn.get(x,0))

1. Split the data into training and test set.
2. Write different logistic regression models predicting y against X.
3. Construct [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) on the test data set for these different models.
4. Analyze these models. Explain which model is the best model you have found.
5. Repeat Steps 1-4 several times. Does your best model stay as the best model? What should be the correct protocol to decide on the best model explaining the data?

In [98]:
X = X.astype(float)

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

***

In [100]:
model1 = LogisticRegression(max_iter=1500)
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.7861271676300579

In [101]:
y_predict = model1.predict(X_test)
confusion_matrix(y_test, y_predict)

array([[82, 11],
       [26, 54]], dtype=int64)

***

In [102]:
df = pd.concat([X_train, y_train], axis=1)
df.columns = ['a','b','c','d','e','p']

In [103]:
model2 = smf.logit('p ~ a + b + c + d + e', data=df).fit()
model2.summary()

Optimization terminated successfully.
         Current function value: 0.501980
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      p   No. Observations:                  517
Model:                          Logit   Df Residuals:                      511
Method:                           MLE   Df Model:                            5
Date:                Fri, 04 Nov 2022   Pseudo R-squ.:                  0.2679
Time:                        04:05:58   Log-Likelihood:                -259.52
converged:                       True   LL-Null:                       -354.51
Covariance Type:            nonrobust   LLR p-value:                 3.962e-39
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.6911      0.308     -5.494      0.000      -2.294      -1.088
a              0.0043      0.009      0.467      0.641      -0.014       0.022
b              0.0335      0.026      1.300      0.194      -0.017       0.084
c              0.1808      0.043      4.175      0.000       0.096       0.266
d              0.3283      0.051      6.449      0.000       0.228       0.428
e              0.0006      0.000      3.863      0.000       0.000       0.001
==============================================================================
"""

***

In [104]:
model3 = sm.Logit(y_train, X_train).fit()
model3.summary()

Optimization terminated successfully.
         Current function value: 0.533356
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     15   No. Observations:                  517
Model:                          Logit   Df Residuals:                      512
Method:                           MLE   Df Model:                            4
Date:                Fri, 04 Nov 2022   Pseudo R-squ.:                  0.2222
Time:                        04:05:58   Log-Likelihood:                -275.75
converged:                       True   LL-Null:                       -354.51
Covariance Type:            nonrobust   LLR p-value:                 4.956e-33
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
1             -0.0397      0.005     -7.296      0.000      -0.050      -0.029
2             -0.0040      0.024     -0.165      0.869      -0.051       0.043
7              0.1894      0.042      4.515      0.000       0.107       0.272
10             0.3071      0.048      6.415      0.000       0.213       0.401
14             0.0006      0.000      3.611      0.000       0.000       0.001
==============================================================================
"""

In [105]:
y_predict3 = model3.predict(X_test)
y_predict3 = np.where(y_predict3 > 0.5, 1, 0)
confusion_matrix(y_test, y_predict3)

array([[78, 15],
       [24, 56]], dtype=int64)

***

In [106]:
model4 = DecisionTreeRegressor()
model4.fit(X_train, y_train)
bootstrap(X, y, model4)

(-0.46256038647343, -0.019009370816599258)

In [115]:
y_predict4 = model4.predict(X_test)
confusion_matrix(y_test, y_predict4)

array([[88,  5],
       [ 6, 74]], dtype=int64)

***

In [119]:
model5 = LinearRegression()
model5.fit(X_train, y_train)
bootstrap(X, y, model5)

(0.003166956460089132, 0.2792715471037618)

In [ ]:
y_predict5 = model5.predict(X_test)
y_predict5 = np.where(y_predict5 > 0.5, 1, 0)
confusion_matrix(y_test, y_predict5)

array([[95,  5],
       [34, 39]], dtype=int64)

***

Binary logistic regression model is the best model because y has 2 values (1 and 0). DecisionTreeRegressor() predicts y better than the others.